# Table selection

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from utils.table_selection.table_selector import *
from utils.table_selection.table_details import *
from utils.api_data_request.api_generator import *

In [3]:
table_descriptions = get_table_schemas()
print(table_descriptions, "\n")

table name: Consumer Price Index - CPI
table description: Table 'Consumer Price Index - CPI' has price index for products, encompassing over 200 expenditure categories grouped into major segments (food and beverages, housing, apparel, recreation, and other goods).
table columns: Year [INTEGER] (year), Consumer Price Inde [FLOAT] (consumer price index), Standard Error [FLOAT] (standard error or CPI), Percent Change [FLOAT] (percentage change), Level 1.1 [TEXT] (enums `PRODUCTS_1`, Level 1 classification of products), Level 2.1 [TEXT] (Level 2.1 classification of products/services)


table name: dot_faf
table description: Table 'dot_faf' has freight movement among states and major metropolitan areas by all modes of transportation. Shows which goods are shipped from one region of the US to another region, according to type of commodity, mode of shipment, value, and weight.
table columns: Year [INTEGER] (year), Millions Of Dollars [FLOAT] (value in millions of dollars of a certain shipment

In [4]:
get_table_names()

['Consumer Price Index - CPI', 'dot_faf', 'ed_defaults', 'pums_5']

In [5]:
query = 'how much coal was moved between New York and California in 2020?'

#### Getting table from LLM (OpenAI's API)

In [6]:
table_llm = get_relevant_tables_from_lm(query)


Messages:
 [{'role': 'user', 'content': 'What service presented the highest growth in the consumer price index during the last year?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Services"]\n}\n'}, {'role': 'user', 'content': 'What is the average monthly CPI volatility for all product groups combined?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'Which states or metropolitan areas are the primary origins for specific products, and which are the primary destinations?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volume across all transportation modes over the past five years?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["dot_faf", "us_location_data"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volu

In [7]:
print(table_llm)


['dot_faf']


#### Getting table from DB (with embeddings)

In [8]:
table_db = get_relevant_tables_from_database(query)

DF:   table_name  similarity
0    dot_faf    0.204381


In [9]:
get_table_schemas(table_db)

"table name: dot_faf\ntable description: Table 'dot_faf' has freight movement among states and major metropolitan areas by all modes of transportation. Shows which goods are shipped from one region of the US to another region, according to type of commodity, mode of shipment, value, and weight.\ntable columns: Year [INTEGER] (year), Millions Of Dollars [FLOAT] (value in millions of dollars of a certain shipment.), Thousands Of Tons [FLOAT] (weight in thousands of tons of a certain shipment.), Origin State [TEXT] (origin state), Origin Region [TEXT] (origin region), Destination State [TEXT] (destination state), Destination Region [TEXT] (destination region), SCTG2 [TEXT] (products based on SCTG classification (first level).), Transportation Mode [TEXT] (mode of transportation or shipment.)\n"

#### Getting table with hybrid approach 
- Gets top k matches from DB, then asks the LLM to choose the most relevant

In [10]:
table_db_llm = request_tables_to_lm_from_db(query)

DF:                    table_name  similarity
0                     dot_faf    0.204381
1                      pums_5    0.181877
2  Consumer Price Index - CPI    0.109880
Messages:
 [{'role': 'user', 'content': 'What service presented the highest growth in the consumer price index during the last year?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Services"]\n}\n'}, {'role': 'user', 'content': 'What is the average monthly CPI volatility for all product groups combined?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'Which states or metropolitan areas are the primary origins for specific products, and which are the primary destinations?'}, {'role': 'assistant', 'content': '\n{\n    "tables": ["Consumer Price Index - Products"]\n}\n'}, {'role': 'user', 'content': 'What is the trend in total freight movement volume across all transportation modes over the past five years?'}

# API parameter generator

In [11]:
v, m, c = get_api_params_from_lm(query, table_db_llm[0], model = 'gpt-4-1106-preview')


ChatGPT response: The chosen variables, measures and filters can answer the query because they directly correspond to the information requested in the query: the amount of coal moved (measure), between New York and California (variables), in 2020 (filter).

The variables "Origin State" and "Destination State" exist in the table and will be used to specify the locations New York and California. The measure "Thousands Of Tons" exists in the table and will be used to quantify the amount of coal moved. The filter "Year = 2020" exists in the table and will be used to specify the time frame of interest. Additionally, the filter "SCTG2 = Coal" will be used to specify the type of commodity, which is also present in the table.

```json
{
    "variables": ["Origin State", "Destination State"],
    "measures": ["Thousands Of Tons"],
    "filters": ["Year = 2020", "SCTG2 = Coal", "Origin State = New York", "Destination State = California"]
}
```

Parameters: {
    "variables": ["Origin State", "D

In [12]:
print("Variables:\n\n", v, "\n", "\nMeasures:\n\n", m, "\n", "\nCuts:\n\n", c)

Variables:

 ['Origin State', 'Destination State'] 
 
Measures:

 ['Thousands Of Tons'] 
 
Cuts:

 ['Year = 2020', 'SCTG2 = Coal', 'Origin State = New York', 'Destination State = California']


In [13]:
api_url = api_build(table = table_db_llm[0], drilldowns = v, measures = m, cuts = c)

In [14]:
api_url

'https://api-ts-zircon.datausa.io/tesseract/data.jsonrecords?cube=dot_faf&&drilldowns=Origin+State,Destination+State&measures=Thousands+Of+Tons&Year=2020&SCTG2=15&Origin State=04000US36&Destination State=04000US06'

# API Request

In [12]:
data, df = api_request(api_url)

In [13]:
data

[{'Origin State ID': '04000US01',
  'Origin State': 'Alabama',
  'Thousands Of Tons': 7598362.334556773},
 {'Origin State ID': '04000US02',
  'Origin State': 'Alaska',
  'Thousands Of Tons': 1239551.9880173053},
 {'Origin State ID': '04000US04',
  'Origin State': 'Arizona',
  'Thousands Of Tons': 3687145.6205386315},
 {'Origin State ID': '04000US05',
  'Origin State': 'Arkansas',
  'Thousands Of Tons': 4779858.874449888},
 {'Origin State ID': '04000US06',
  'Origin State': 'California',
  'Thousands Of Tons': 23684891.263292156},
 {'Origin State ID': '04000US08',
  'Origin State': 'Colorado',
  'Thousands Of Tons': 5810747.717232299},
 {'Origin State ID': '04000US09',
  'Origin State': 'Connecticut',
  'Thousands Of Tons': 2391268.6159814},
 {'Origin State ID': '04000US10',
  'Origin State': 'Delaware',
  'Thousands Of Tons': 923191.2903833825},
 {'Origin State ID': '04000US11',
  'Origin State': 'District of Columbia',
  'Thousands Of Tons': 122755.04869312332},
 {'Origin State ID': '

In [10]:
df

,ID Year,Year,Average Wage
0,2021,2021,27936.525449
1,2020,2020,26200.581324
2,2019,2019,25356.424307
3,2018,2018,24316.619758
4,2017,2017,23247.646201
5,2016,2016,22225.811092
6,2015,2015,21547.164279
7,2014,2014,21282.953902
